# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
# import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
# from data_processing import DataProcessing

In [2]:
# load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.9796735644340515

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8603723645210266

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8691259026527405

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8487551808357239

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8647664785385132

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8541026711463928

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8756678104400635



/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_88690/312081125.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


In [5]:
# pom_mappings

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.979674
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.860372
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.869126
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.848755
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.864766
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.854103
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.875668


Comparison
1. No focus on #s, more on words

In [7]:
prediction_doc = load_nlp_model("10%")
observation_doc = load_nlp_model("10.5%")
observation_doc_2 = load_nlp_model("11%")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_88690/1801851099.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.9251048564910889

In [8]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_88690/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.9595414400100708

In [9]:
prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
observation_doc = load_nlp_model("Apple's stock rose by 11%.")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_88690/3994610451.py:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.9899832606315613

In [ ]:
pdv = prediction_doc.vector # 10
pdv

array([-9.69052166e-02,  4.43707854e-02, -3.03838477e-02, -2.54174694e-04,
        3.76424938e-01, -8.68998468e-03,  3.35592031e-01,  2.44690776e-01,
        5.41072249e-01, -3.75136226e-01, -5.61025739e-02,  4.03797567e-01,
       -3.96669120e-01,  6.21899724e-01, -1.49399251e-01,  4.20594633e-01,
       -6.60013497e-01,  4.23254371e-02,  1.75270349e-01,  2.39882201e-01,
       -3.27159762e-01,  2.70389676e-01,  1.91664666e-01, -1.41071290e-01,
        4.56980765e-01, -2.14351714e-02,  1.68063223e-01, -1.47116721e-01,
        2.89462171e-02,  2.59644151e-01,  1.40097201e-01, -3.19258392e-01,
        1.61395982e-01, -5.15149355e-01, -2.74461329e-01,  1.34547725e-01,
       -1.52158603e-01, -2.81526130e-02, -8.63277316e-02,  2.68475622e-01,
        2.45270133e-01, -6.46607578e-03,  2.96593577e-01,  4.67894614e-01,
        7.09100813e-02, -3.24479461e-01, -6.74477741e-02,  6.88518107e-01,
        1.01036236e-01, -3.16862434e-01,  7.60792196e-02,  7.62369931e-02,
        3.15949544e-02, -

In [ ]:
odv = observation_doc.vector # 10.5
odv

array([-0.1375021 ,  0.05036654, -0.0824471 , -0.09218727,  0.42592007,
       -0.02057144,  0.3527472 ,  0.24258192,  0.588894  , -0.40792432,
       -0.0078516 ,  0.41594055, -0.30752772,  0.6503702 , -0.10828097,
        0.39248452, -0.726854  ,  0.08076884,  0.20747659,  0.31021383,
       -0.31106994,  0.31786215,  0.10419345, -0.14996357,  0.45800018,
        0.00623147,  0.10050573, -0.1621193 ,  0.03166858,  0.29230028,
        0.16873193, -0.37811226,  0.21399517, -0.57316726, -0.24211697,
        0.13694435, -0.1441578 , -0.05136697, -0.07773665,  0.2569706 ,
        0.31111842, -0.05671661,  0.27724874,  0.45298147,  0.06111439,
       -0.29012552, -0.06525761,  0.67409015,  0.1075652 , -0.33304486,
       -0.01102823,  0.11045703, -0.02698915, -0.6804062 ,  0.29783314,
       -0.1287547 ,  0.17033227, -0.34667575,  0.56826645, -0.13759255,
       -0.5063458 , -0.75957036,  0.00584174, -0.26786044, -0.09881489,
       -0.14349848,  0.46282774, -0.46586582, -0.25601786,  0.21

[Five most popular similarity measures implementation in python](https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/)

In [12]:
from math import*

def square_rooted(x):

    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):

    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

cosine_similarity(pdv, odv)

np.float32(0.99)